# 第2回
- モデルのパラメータをいじってみる
- early_stoppingで最適な学習回数まで学習を行う
- 特徴量の重要度を可視化してみる

In [ ]:
# 使用するライブラリ(numpy, pandas, matplotlib.pyplot)をインポートしましょう
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## データの読み込みと確認

In [ ]:
# 受領した各データ(train.csv, test.csv, sample_submission.csv)を読み込みましょう
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')
sample_submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

In [ ]:
# trainデータの先頭5行を確認しましょう
train.head()

## データの加工
- ここはいくらでも手の加えようがあるので、自身で思いついた加工内容に変更してしまってOKです
- Cabin, PassengerIdから特徴量が作れないか検討してみましょう
- EDAから仮説を立てて、効きそうな特徴量を検討してみましょう
- 正解コードは簡易な書き方をしており、繰り返しが多いので、関数化できたらしてしまった方が楽かもしれません

In [ ]:
# bool型(True/False)の変数を1/0に変換しましょう
train['CryoSleep'] = train['CryoSleep'].astype(float)
test['CryoSleep'] = test['CryoSleep'].astype(float)

train['VIP'] = train['VIP'].astype(float)
test['VIP'] = test['VIP'].astype(float)

train['Transported'] = train['Transported'].astype(float)

In [ ]:
# 分析に利用できそうなカテゴリ変数(文字型)を数値に変換しましょう
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['HomePlanet'] = le.fit_transform(train['HomePlanet'])
test['HomePlanet'] = le.transform(test['HomePlanet'])

le = LabelEncoder()
train['Destination'] = le.fit_transform(train['Destination'])
test['Destination'] = le.transform(test['Destination'])

In [ ]:
# 分析に用いない列を削除しましょう
train = train.drop(['PassengerId', 'Cabin', 'Name'], axis=1)
test = test.drop(['PassengerId', 'Cabin', 'Name'], axis=1)

In [ ]:
train.head()

## バリデーションデータの作成

In [ ]:
# trainデータセットを、説明変数と目的変数に分割しましょう
X = train.drop('Transported', axis=1)
y = train['Transported']

In [ ]:
# train_test_split関数を用いて、データセットを訓練用と評価用に分割しましょう
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=0
)


## モデルの学習
- ここでは分析コンペでよく用いられるLightGBMを用います
- 他モデルを用いても良いですが、その場合以下のような加工が必要になります
     - nullを埋める
     - カテゴリ変数をone-hot encodingする(決定木系のモデルでない場合)
     - 標準化する(線形モデルやニューラルネットワークの場合)

In [ ]:
# 参考：https://blog.amedama.jp/entry/2018/05/01/081842
# 回答はscikit_learnインターフェースでない記法で書いております（パラメータチューニングやearly_stoppingなど、ググると日本語の記事が多くヒットする記法なため）
# lightgbmをインポートしましょう
import lightgbm as lgb

# データセットを生成しましょう
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

# lightGBMのパラメータを辞書型で定義しましょう（参考：https://qiita.com/nabenabe0928/items/6b9772131ba89da00354）
# lightGBMはパラメータ調整しなくても割といいスコアが出るので、「こんな書き方をするんだ」くらいに知っておけばOKです
lgb_params = {
    'objective': 'binary', 
    'metric': 'binary_logloss', 
    'verbosity': -1
}

# lightGBMを学習しましょう(early_stoppingをかけるようにコードを書いてみてください)
model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_valid, num_boost_round=1000, early_stopping_rounds=10)


In [ ]:
# scikit-learn APIとそうでない記法では、出力の形が異なるため注意
# scikit-learn APIは予測のラベル(1/0)が、そうでない場合はラベルが1である予測確率が出力される
model.predict(X_valid)

## モデルの評価

In [ ]:
# 評価用データから、予測を出力しましょう
y_pred = model.predict(X_valid)
# scikit-learn APIでない記法を用いている場合、予測確率0.5以上を1, そうでないものを0として予測を作成しましょう
y_pred = np.where(y_pred>=0.5, 1, 0)

# accuracy_score関数を用いて、スコアを計算しましょう
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_valid)

In [ ]:
# 特徴量の重要度をプロットしてみる
lgb.plot_importance(model, figsize=(12,6))
plt.show()

## 提出用ファイルの作成

In [ ]:
# sample_submissionのTransported列を上書きする形で予測を作成します(他コンペも概ねその形を取ります)
# sample_submissionの先頭行を見て、形式を確認しましょう
sample_submission.head()

In [ ]:
# sample_submissionのTransported列を、モデルの予測に置き換えましょう
# 予測は1/0でなされているため、bool型に変換するのを忘れないように
y_pred = model.predict(test)
y_pred = np.where(y_pred>=0.5, 1, 0)
sample_submission['Transported'] = y_pred.astype(bool)
sample_submission.head()

In [ ]:
# csvとして出力（indexがファイル出力に含まれていると、提出形式に沿わなくなるため注意）
sample_submission.to_csv('submission.csv', index=False)